# Generating Populations and Species
Assuming that the civilization is on the verge of moving from a phaze 0 to phase 1 civ.
* baseline species
* n populations (pops)
* population varience (within the norms of the species)
* allegiance groups to factions

Starting with an example user form data

In [28]:

from numpy import random, round
from sklearn.cluster import KMeans

import pandas as pd


In [20]:

data = {
    'population_conformity':0.4,
    'population_literacy':0.7,
    'population_aggression':0.5,
    'population_constitution':0.4,
    'starting_pop': 7
}

def build_species(data):
    species = {}
    for attr in ['population_conformity', 'population_literacy', 'population_aggression', 'population_constitution']:
        species[attr] = data[attr]
    return species
    
species = build_species(data)
species


{'population_conformity': 0.4,
 'population_literacy': 0.7,
 'population_aggression': 0.5,
 'population_constitution': 0.4}

In [29]:
pop_std = .2

def vary_pops(species):
    pop = {}
    for k in list(species.keys()):
        pop[k] = abs(round(random.normal(species[k], pop_std),3))
    return pop

pd.DataFrame([vary_pops(species) for i in range(data['starting_pop'])])

,population_conformity,population_literacy,population_aggression,population_constitution
0,0.565,0.099,0.502,0.406
1,0.354,0.726,0.332,0.501
2,0.498,0.935,0.512,0.310
3,0.240,0.883,0.537,0.496
4,0.202,0.572,0.502,0.460
5,0.510,1.005,0.047,0.218
6,0.405,0.847,0.357,0.586
